# Preview all the scans we did
Generate MIPs and get a quick look on the details.

In [ ]:
import platform
import os
import glob
import pandas
import imageio
import numpy
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
import seaborn
import dask
import dask_image.imread
from dask.distributed import Client, LocalCluster
from numcodecs import Blosc
import skimage
from tqdm.auto import tqdm

In [ ]:
from BrukerSkyScanLogfileRuminator.parsing_functions import *

In [ ]:
# Set dask temporary folder
# Do this before creating a client: https://stackoverflow.com/a/62804525/323100
import tempfile
if 'Linux' in platform.system():
    tmp = os.path.join(os.sep, 'media', 'habi', 'FastSSD')
elif 'Darwin' in platform.system():
    tmp = tempfile.gettempdir()
else:
    if 'anaklin' in platform.node():
        tmp = os.path.join('F:\\')
    else:
        tmp = os.path.join('D:\\')
dask.config.set({'temporary_directory': os.path.join(tmp, 'tmp')})
print('Dask temporary files go to %s' % dask.config.get('temporary_directory'))

In [ ]:
# Start cluster and client now, after setting tempdir
cluster = LocalCluster()
client = Client(cluster)

In [ ]:
print('You can seee what DASK is doing at "http://localhost:%s/status"' % client.scheduler_info()['services']['dashboard'])

In [ ]:
# Set up figure defaults
plt.rc('image', cmap='gray', interpolation='nearest')  # Display all images in b&w and with 'nearest' interpolation
plt.rcParams['figure.figsize'] = (16, 9)  # Size up figures a bit
plt.rcParams['figure.dpi'] = 200  # Increase dpi

In [ ]:
# Setup scale bar defaults
plt.rcParams['scalebar.location'] = 'lower right'
plt.rcParams['scalebar.frameon'] = False
plt.rcParams['scalebar.color'] = 'white'

In [ ]:
# Display all plots identically
lines = 2
# And then do something like
# plt.subplot(lines, numpy.ceil(len(Data) / float(lines)), c + 1)

In [ ]:
# Different locations if running either on Linux or Windows
if 'anaklin' in platform.node():
    FastSSD = True
else:
    FastSSD = False
# to speed things up significantly
if 'Linux' in platform.system():
    if FastSSD:
        BasePath = os.path.join(os.sep, 'media', 'habi', 'FastSSD')
    else:
        BasePath = os.path.join(os.sep, 'home', 'habi', '1272')
elif 'Darwin' in platform.system():
    BasePath = os.path.join('/Volumes/2TBSSD/')
else:
    if FastSSD:
        BasePath = os.path.join('F:\\')
    else:
        if 'anaklin' in platform.node():
            BasePath = os.path.join('S:\\')
        else:
            BasePath = os.path.join('D:\\', 'Results')
Root = os.path.join(BasePath, 'Hearts Melly')
print('We are loading all the data from %s' % Root)

In [ ]:
# Make us a dataframe for saving all that we need
Data = pandas.DataFrame()

In [ ]:
# Get *all* log files, unsorted but fast
Data['LogFile'] = [os.path.join(root, name)
                   for root, dirs, files in os.walk(Root)
                   for name in files
                   if name.endswith((".log"))]

In [ ]:
Data

In [ ]:
# Get all folders
Data['Folder'] = [os.path.dirname(f) for f in Data['LogFile']]

In [ ]:
# Check for samples which are not yet reconstructed
for c, row in Data.iterrows():
    # Iterate over every 'proj' folder
    if 'proj' in row.Folder:
        if not 'TScopy' in row.Folder and not 'PR' in row.Folder:
            # If there's nothing with 'rec*' on the same level, then tell us        
            if not glob.glob(row.Folder.replace('proj', '*rec*')):
                print('- %s is missing matching reconstructions' % row.LogFile[len(Root)+1:])

In [ ]:
# Search for any .csv files in each folder
Data['XYAlignment'] = [glob.glob(os.path.join(f, '*T*.csv')) for f in Data['Folder']]

In [ ]:
# Display samples which are missing the .csv-files for the XY-alignment
for c, row in Data.iterrows():
    # Iterate over every 'proj' folder
    if 'proj' in row['Folder']:
        if not row['XYAlignment']:
            if not any(x in row.LogFile for x in ['rectmp.log',  # because we only exclude temporary logfiles in a later step
                                                  #'proj_nofilter',  # since these two scans of single teeth don't contain a reference scan
                                                  #'TScopy',  # discard *t*hermal *s*hift data
                                                  ]):
                print('- %s has *not* been X/Y aligned' % row.LogFile[len(Root) + 1:])

In [ ]:
# Get rid of all non-rec logfiles
for c, row in Data.iterrows():
    if 'rec' not in row.Folder:
        Data.drop([c], inplace=True)
    elif 'ctan.log' in row.LogFile:
        Data.drop([c], inplace=True)
    elif 'rectmp.log' in row.LogFile:
        Data.drop([c], inplace=True)
# Reset dataframe to something that we would get if we only would have loaded the 'rec' files
Data = Data.reset_index(drop=True)

In [ ]:
# Drop all folders we don't need
for c, row in Data.iterrows():
    if 'Rat' not in row.Folder:
        Data.drop([c], inplace=True)
#     elif 'overview' not in row.Folder:
#         Data.drop([c], inplace=True)
    elif 'Test' in row.Folder:
        Data.drop([c], inplace=True)
# Reset dataframe to something that we would get if we only would have loaded the 'rec' files
Data = Data.reset_index(drop=True)

In [ ]:
# Generate us some meaningful colums
Data['Animal'] = [l[len(Root)+1:].split(os.sep)[0] for l in Data['LogFile']]
Data['Scan'] = ['.'.join(l[len(Root)+1:].split(os.sep)[1:-1]) for l in Data['LogFile']]

In [ ]:
print('We habe %s scans to work with' % len(Data))

In [ ]:
# Read in animals list from Ludovic
AnimalTable = pandas.read_excel('Animals.xlsx',
                                engine='openpyxl',
                                header=None,
                                names=['Animal', 'Gender', '', 'Experiment', 'Timepoint'])

In [ ]:
# Merge in data from animals table
for c, rowdata in Data.iterrows():
    for d, rowanimals in AnimalTable.iterrows():
        if str(rowanimals.Animal) in rowdata.Animal:
            Data.at[c, 'Experiment'] = rowanimals.Experiment
            Data.at[c, 'Timepoint'] = rowanimals.Timepoint
            Data.at[c, 'Gender'] = rowanimals.Gender

In [ ]:
# Now that we merged the data we can rename the column to a more reusable name
Data.columns = Data.columns.str.replace('Animal', 'Sample')

In [ ]:
# Get parameters to doublecheck from logfiles
Data['Voxelsize'] = [pixelsize(log) for log in Data['LogFile']]
Data['Filter'] = [whichfilter(log) for log in Data['LogFile']]
Data['Exposuretime'] = [exposure(log) for log in Data['LogFile']]
Data['Grayvalue'] = [reconstruction_grayvalue(log) for log in Data['LogFile']]
Data['RingartefactCorrection'] = [ringremoval(log) for log in Data['LogFile']]
Data['BeamHardeningCorrection'] = [beamhardening(log) for log in Data['LogFile']]

In [ ]:
# Did we reconstruct them equally?
Data[['Sample', 'Scan', 'Voxelsize', 'Filter', 'Exposuretime', 'Grayvalue', 'RingartefactCorrection', 'BeamHardeningCorrection']]

In [ ]:
print(Data['Voxelsize'].unique())
print(Data['Exposuretime'].unique())
print(Data['Filter'].unique())
print(Data['Grayvalue'].unique())
print(Data['RingartefactCorrection'].unique())
print(Data['BeamHardeningCorrection'].unique())

In [ ]:
# Save to XLS sheet for Ludovic
Data[['Folder', 'Sample', 'Scan', 'Voxelsize']].to_excel('Voxelsizes.xlsx')

In [ ]:
# Get the file names of the reconstructions
Data['Reconstructions'] = [sorted(glob.glob(os.path.join(f,
                                                         '*.png'))) for f in Data['Folder']]
Data['Number of reconstructions'] = [len(r) for r in Data.Reconstructions]

In [ ]:
Data

In [ ]:
# Drop samples which have not been reconstructed yet
# Based on https://stackoverflow.com/a/13851602
for c, row in Data[Data['Number of reconstructions'] == 0].iterrows():
    print('%s/%s has not been reconstructed yet, we remove it from our data temporarily' % (row.Sample, row.Scan))
    Data = Data[Data['Number of reconstructions'] > 0]
    Data.reset_index(drop=True, inplace=True)
print('We have %s folders with reconstructions' % (len(Data)))

In [ ]:
# # Display some info
# for c, row in Data.iterrows():
#     print('%02s/%s: %s has %03s reconstructions in %s/%s' % (c + 1,
#                                                              len(Data),
#                                                              row['Sample'].rjust(Data['SampleNameLength'].max()),
#                                                              row['Number of reconstructions'],
#                                                              os.path.basename(row['Folder']),
#                                                              'rec'))

In [ ]:
# Data['PreviewImagePath'] = [sorted(glob.glob(os.path.join(f, '*_spr.bmp')))[0] for f in Data['Folder']]
# # Data['PreviewImage'] = [imageio.imread(pip)
# #                         if pip
# #                         else numpy.random.random((100, 100)) for pip in Data['PreviewImagePath']]

In [ ]:
# Load all (original, uncropped) reconstructions into ephemereal DASK arrays
Reconstructions = [None] * len(Data)
for c, row in tqdm(Data.iterrows(), desc='Load reconstructions', total=len(Data)):
    Reconstructions[c] = dask_image.imread.imread(os.path.join(row['Folder'], '*rec*.png'))[:,:,:,0].rechunk('auto')

In [ ]:
Reconstructions[0]

In [ ]:
# The three cardinal directions
directions = ['Axial',
              'Coronal',
              'Sagittal']

In [ ]:
def cropper(image,
            despeckle=True,
            verbose=False,
            threshold=50):
    import skimage.morphology
    import scipy
    ''' Function to crop the reconstruction stack to its 'minimal' size, based on the biggest item visible in the MIP '''
    dimensions = len(image.shape)
    if verbose:
        print('Cropping %s-dimensional image' % dimensions)
    # Threshold
    thresholded = image > threshold
    if despeckle:
        if verbose:
            print('Removing small objects')
        despeckled = skimage.util.apply_parallel(skimage.morphology.remove_small_objects,
                                                 thresholded,
                                                 extra_keywords={'min_size': 10**dimensions},
                                                 chunks=[15,15,15])
    # Find biggest object
    # https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.find_objects.html
    if verbose:
        print('Finding objects')
    if despeckle:
        cropdimensions = scipy.ndimage.find_objects(despeckled)[0]
    else:
        cropdimensions = scipy.ndimage.find_objects(thresholded)[0]
    if verbose:
        print('Cutting the image down to')
        for c, cd in enumerate(cropdimensions):
            print('\t- %s: %s' % (c, cd))
        if dimensions > 2:
            # Calculate cardinal direction MIPs
            MIPs = [image.max(axis=d) for d, direction in tqdm(enumerate(directions),
                                                                                  desc='Calculating MIPs',
                                                                                  total=len(directions))]
            for d, direction in enumerate(directions):
                plt.subplot(1, dimensions, d + 1)
                plt.imshow(MIPs[d], alpha=0.618)
                plt.imshow(dask.array.ma.masked_less(MIPs[d]>threshold, 1),
                           alpha=0.309,
                           cmap='viridis_r')
                if d == 0:
                    plt.axhline(cropdimensions[1].start,
                                c=seaborn.color_palette()[0],
                                label=cropdimensions[1].start)
                    plt.axhline(cropdimensions[1].stop,
                                c=seaborn.color_palette()[1],
                                label=cropdimensions[1].stop)
                    plt.axvline(cropdimensions[2].start,
                                c=seaborn.color_palette()[2],
                                label=cropdimensions[2].start)
                    plt.axvline(cropdimensions[2].stop,
                                c=seaborn.color_palette()[3],
                                label=cropdimensions[2].stop)
                elif d == 1:
                    plt.axhline(cropdimensions[0].start,
                                c=seaborn.color_palette()[0],
                                label=cropdimensions[0].start)
                    plt.axhline(cropdimensions[0].stop,
                                c=seaborn.color_palette()[1],
                                label=cropdimensions[0].stop)
                    plt.axvline(cropdimensions[2].start,
                                c=seaborn.color_palette()[2],
                                label=cropdimensions[2].start)
                    plt.axvline(cropdimensions[2].stop,
                                c=seaborn.color_palette()[3],
                                label=cropdimensions[2].stop)                    
                elif d == 2:
                    plt.axhline(cropdimensions[0].start,
                                c=seaborn.color_palette()[0],
                                label=cropdimensions[0].start)
                    plt.axhline(cropdimensions[0].stop,
                                c=seaborn.color_palette()[1],
                                label=cropdimensions[0].stop)
                    plt.axvline(cropdimensions[1].start,
                                c=seaborn.color_palette()[2],
                                label=cropdimensions[1].start)
                    plt.axvline(cropdimensions[1].stop,
                                c=seaborn.color_palette()[3],
                                label=cropdimensions[1].stop)                                        
                plt.title('%s MIP' % direction)
                plt.legend(loc='lower right')
        else:
            plt.subplot(121)
            plt.imshow(image, alpha=0.618)
            plt.imshow(dask.array.ma.masked_less(thresholded, 1), alpha=0.618, cmap='viridis_r')
            plt.axhline(cropdimensions[0].start, c=seaborn.color_palette()[0], label=cropdimensions[0].start)
            plt.axhline(cropdimensions[0].stop, c=seaborn.color_palette()[1], label=cropdimensions[0].stop)
            plt.axvline(cropdimensions[1].start, c=seaborn.color_palette()[2], label=cropdimensions[1].start)
            plt.axvline(cropdimensions[1].stop, c=seaborn.color_palette()[3], label=cropdimensions[1].stop)
            plt.title('Original image with thresholded overlay')
            plt.legend(loc='lower right')
            plt.subplot(122)
            plt.imshow(image[cropdimensions])
            plt.title('Output')
        plt.show()
    if verbose:
        print('Cropped image from %s to %s' % (image.shape, image[cropdimensions].shape))
        print(cropdimensions)
    return(image[cropdimensions])

In [ ]:
# # Test cropping function
# img = Reconstructions[-1]
# Cropped = cropper(img, verbose=True, despeckle=True, threshold=55)
# for c, direction in enumerate(directions):
#     plt.subplot(1,3,c+1)    
#     if 'Axial' in direction:
#         plt.imshow(Cropped[Cropped.shape[0] // 2])
#     if 'Sagittal' in direction:
#         plt.imshow(Cropped[:, Cropped.shape[1] // 2, :])
#     if 'Coronal' in direction:
#         plt.imshow(Cropped[:, :, Cropped.shape[2] // 2])

In [ ]:
# cropper(Reconstructions[3], verbose=True)

In [ ]:
# Crop the reconstructions and save them out as .zarr files
Data['OutputNameRecCrop'] = [os.path.join(os.path.dirname(f),
                                          '%s.%s.crop.zarr' % (sample, scan)) for f, sample, scan in zip(Data.Folder,
                                                                                                         Data.Sample,
                                                                                                         Data.Scan)]
for c, row in tqdm(Data.iterrows(),
                            desc='Cropping reconstructions and saving to rechunked .zarr files',
                            total=len(Data)):
    if not os.path.exists(row['OutputNameRecCrop']):
        print('%2s/%2s: Saving to %s' % (c + 1,
                                         len(Data),
                                         row['OutputNameRecCrop'][len(Root):]))
        # Calculate crop with our function and directly write to ZARR file
        cropper(Reconstructions[c],
                verbose=True,
                despeckle=True).rechunk(chunks='auto').to_zarr(row['OutputNameRecCrop'],
                                                               overwrite=True) 

In [ ]:
# Load the *cropped* zarr arrays as reconstructions
Reconstructions = [dask.array.from_zarr(file) for file in Data['OutputNameRecCrop']]

In [ ]:
for r in Reconstructions:
    if len(r.shape) != 3:
        print(r.shape)

In [ ]:
# # Save out cropped rec slices
# for c, row in tqdm(Data.iterrows(),
#                             desc='Saving out cropped recs',
#                             total=len(Data)):
#     os.makedirs(os.path.join(row.Folder, 'rec_crop'),
#                 exist_ok=True)
#     for d, rec in tqdm(enumerate(Reconstructions[c]),
#                                 total=len(Reconstructions[c]),
#                                 desc=os.path.join(row.Sample, os.path.basename(row.Folder)),
#                                 leave=False):
#         filename = os.path.join(row.Folder,
#                                 'rec_crop', str(row.Sample) + '_rec_crop_%08d.png' % d)
#         if not os.path.exists(filename):
#             imageio.imsave(filename, rec)

In [ ]:
# How big are the datasets?
Data['Size'] = [numpy.shape(rec) for rec in Reconstructions]

In [ ]:
# Read or calculate the middle slices, put them into the dataframe and save them to disk
for d, direction in enumerate(directions):
    Data['Mid_' + direction] = [None] * len(Reconstructions)
for c, row in tqdm(Data.iterrows(), desc='Middle images', total=len(Data)):
    for d, direction in tqdm(enumerate(directions),
                                      desc='%s/%s' % (row['Sample'], row['Scan']),
                                      leave=False,
                                      total=len(directions)):
        outfilepath = os.path.join(os.path.dirname(row['Folder']),
                                   '%s.%s.Middle.%s.png' % (row['Sample'],
                                                            row['Scan'],
                                                            direction))
        if os.path.exists(outfilepath):
            Data.at[c, 'Mid_' + direction] = dask_image.imread.imread(outfilepath).squeeze()
        else:
            # Generate requested axial view
            if 'Axial' in direction:
                Data.at[c, 'Mid_' + direction] = Reconstructions[c][Data['Size'][c][0] // 2].compute()
            if 'Sagittal' in direction:
                Data.at[c, 'Mid_' + direction] = Reconstructions[c][:, Data['Size'][c][1] // 2, :].compute()
            if 'Coronal' in direction:
                Data.at[c, 'Mid_' + direction] = Reconstructions[c][:, :, Data['Size'][c][2] // 2].compute()
            # Save the calculated 'direction' view to disk
            imageio.imwrite(outfilepath, (Data.at[c, 'Mid_' + direction]))

In [ ]:
# Show middle slices
for c, row in tqdm(Data.iterrows(),
                            desc='Saving middle images overview',
                            total=len(Data)):
    outfilepath = os.path.join(os.path.dirname(row['Folder']),
                               '%s.%s.MiddleSlices.png' % (row['Sample'], row['Scan']))
    if not os.path.exists(outfilepath):    
        for d, direction in tqdm(enumerate(directions),
                                          desc='%s/%s' % (row['Sample'], row['Scan']),
                                          leave=False,
                                          total=len(directions)):
            plt.subplot(1, 3, d + 1)
            plt.imshow(row['Mid_' + direction].squeeze())
            if d == 0:
                plt.axhline(row.Size[1] // 2, c=seaborn.color_palette()[0])
                plt.axvline(row.Size[2] // 2, c=seaborn.color_palette()[1])
                plt.gca().add_artist(ScaleBar(row['Voxelsize'],
                                              'um',
                                              color=seaborn.color_palette()[2]))
            elif d == 1:
                plt.axhline(row.Size[0] // 2, c=seaborn.color_palette()[2])
                plt.axvline(row.Size[d] // 2, c=seaborn.color_palette()[1])
                plt.gca().add_artist(ScaleBar(row['Voxelsize'],
                                              'um',
                                              color=seaborn.color_palette()[0]))
            else:
                plt.axhline(row.Size[0] // 2, c=seaborn.color_palette()[2])
                plt.axvline(row.Size[d] // 2, c=seaborn.color_palette()[0])
                plt.gca().add_artist(ScaleBar(row['Voxelsize'],
                                              'um',
                                              color=seaborn.color_palette()[1]))
            plt.title('%s\n%s' % (os.path.join(row['Sample'], row['Scan']),
                                  direction + ' Middle slice'))
            plt.axis('off')
            plt.savefig(outfilepath,
                        transparent=True,
                        bbox_inches='tight')
        plt.show()

In [ ]:
# Read or calculate the directional MIPs, put them into the dataframe and save them to disk
for d, direction in enumerate(directions):
    Data['MIP_' + direction] = [None] * len(Reconstructions)
for c, row in tqdm(Data.iterrows(), desc='MIPs', total=len(Data)):
    for d, direction in tqdm(enumerate(directions),
                                      desc='%s/%s' % (row['Sample'], row['Scan']),
                                      leave=False,
                                      total=len(directions)):
        outfilepath = os.path.join(os.path.dirname(row['Folder']),
                                   '%s.%s.MIP.%s.png' % (row['Sample'], row['Scan'], direction))
        if os.path.exists(outfilepath):
            Data.at[c, 'MIP_' + direction] = dask_image.imread.imread(outfilepath).squeeze()
        else:
            # Generate MIP
            Data.at[c, 'MIP_' + direction] = Reconstructions[c].max(axis=d).compute()
            # Save it out
            imageio.imwrite(outfilepath, Data.at[c, 'MIP_' + direction].astype('uint8'))

In [ ]:
# Show MIP slices
for c, row in tqdm(Data.iterrows(), desc='Saving MIP images overview', total=len(Data)):
    outfilepath = os.path.join(os.path.dirname(row['Folder']),
                               '%s.%s.MIPs.png' % (row['Sample'], row['Scan']))
    if not os.path.exists(outfilepath):    
        for d, direction in tqdm(enumerate(directions),
                                          desc='%s/%s' % (row['Sample'], row['Scan']),
                                          leave=False,
                                          total=len(directions)):
            plt.subplot(1, 3, d + 1)
            plt.imshow(row['MIP_' + direction].squeeze())
            plt.gca().add_artist(ScaleBar(row['Voxelsize'],
                                          'um'))
            plt.title('%s\n%s' % (os.path.join(row['Sample'], row['Scan']),
                                 direction + ' MIP'))
            plt.axis('off')
        plt.savefig(outfilepath,
                    transparent=True,
                    bbox_inches='tight')
        plt.show()

In [ ]:
# Show MIPs together
lines=5
for d, direction in enumerate(directions):
    for c, row in Data.iterrows():
        plt.subplot(lines, int(numpy.ceil(len(Data) / float(lines))), c + 1)
        plt.imshow(row['MIP_' + direction].squeeze())
        plt.gca().add_artist(ScaleBar(row['Voxelsize'], 'um'))
        plt.title('%s\n%s' % (row['Sample'], row['Scan']))
        plt.axis('off')
        plt.suptitle(direction + ' MIPs')
    plt.savefig(os.path.join(Root, 'MIPs.' + direction + '.png'),
                transparent=True,
                bbox_inches='tight')
    plt.show()

In [ ]:
for i in Data:
    print(i)

In [ ]:
# Calculate the histograms of one of the MIPs
# Caveat: dask.da.histogram returns histogram AND bins, making each histogram a 'nested' list of [h, b]
Data['Histogram'] = [dask.array.histogram(dask.array.array(mip),
                                          bins=2**8,
                                          range=[0, 2**8]) for mip in Data['MIP_Sagittal']]
# Actually compute the data and put only h into the dataframe, so we can use it below.
# Discard the bins
Data['Histogram'] = [h for h, b in Data['Histogram']]

In [ ]:
def overeexposecheck(item,
                     threshold=250,
                     howmanypercent=1,
                     whichone='Lateral',
                     verbose=False):
    '''Function to check if a certain amount of voxels are brighter than a certain value'''
    if (Data['MIP_%s' % whichone][item] > threshold).sum() > (Data['MIP_%s' % whichone][item].size * howmanypercent / 100):
        if verbose:
            plt.subplot(121)
            plt.imshow(Data['MIP_%s' % whichone][item])
            plt.imshow(numpy.ma.masked_equal(Data['MIP_%s' % whichone][item] > threshold, False),
                       cmap='viridis_r',
                       alpha=.618)
            plt.title('%s/%s\n%s px of %s Mpixels (>%s%%) are brighter '
                      'than %s' % (Data['Sample'][item],
                                   Data['Scan'][item],
                                   (Data['MIP_%s' % whichone][item] > threshold).compute().sum(),
                                   round(1e-6 * Data['MIP_%s' % whichone][item].size, 2),
                                   howmanypercent,
                                   threshold))
            plt.axis('off')
            plt.gca().add_artist(ScaleBar(Data['Voxelsize'][item],
                                          'um'))
            plt.subplot(122)
            plt.semilogy(Data['Histogram'][item])
            plt.xlim([0, 2**8])
            plt.show()
        return(True)
    else:
        return(False)

In [ ]:
# Check if 'too much' of the MIP is overexposed
Data['OverExposed'] = [overeexposecheck(c,
                                        whichone='Sagittal',
                                        verbose=True) for c, row in Data.iterrows()]

In [ ]:
# Save mean of reconstruction gray values, which we can use for getting an overview of the image data
#Data['GrayValueMean'] = [rec.mean().compute() for rec in Reconstructions]

In [ ]:
# Save STD of reconstruction gray values, which we can use for getting an overview of the image data
#Data['GrayValueSTD'] = [rec.std().compute() for rec in Reconstructions]

In [ ]:
# Save out the dataframe to disk, so we can use it later in another notebook
#Data.to_pickle(os.path.join(Root, 'MellyDataframe.pkl'))

In [ ]:
#view(Reconstructions[2],
#     annotations=False)

In [ ]:
print('Done with previewing %s scans' % len(Data))